# HARP Labs
## Have Another Retirement Planning Laboratory

This package is a laboratory for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment to explore *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have an asset earnings plan. This is where this tool fits it. Given your savings and spending desires, it can generate different realizations of your strategy under different market assumptions.

Copyright - Martin-D. Lacasse (2023)

Diclaimer: *You make your own decisions and I am not a financial planner. Use at your own risks.*

## Introduction
This file is provided as a template to introduce you to HARP Labs. This notebook describes the case of Jack and Jill, a ficticious couple used for demonstration.

For simulating your own realizations, it is best to make a copy of this file and all those beginning by *jack+jill*. Copy them keeping the same extension and give them you own names. Then you'll be able to personalize a case with your own numbers and start experimenting with HARP Labs.

I assume that you have some familiarity with using a jupyter notebook, and some basic programming skills in Python.

### Just some Python module bookkeeping
These commands load the required module and set the program to display informational messages as it runs. It also makes sure that all graphs are displayed in the jupyter interface.

In [ ]:
%matplotlib inline
# import harp as rp

import importlib
rp = importlib.import_module('harp')
importlib.reload(rp)

rp.setVerbose(True)

## Initialize your realization
In order to be able to generate a realization of the future, Jack and Jill must start with providing the year of birth of each spouse(s) and their expected lifespan(s).

For selecting your own numbers, there are plenty of longivity predictors on the web. Pick your favorite:

https://www.livingto100.com/calculator

https://www.sunlife.ca/en/tools-and-resources/tools-and-calculators/life-expectancy-calculator/

or just Google life expectancy calculator.

There are two values needed for couples. Single individuals just enter one value in each list [ ]. For couples, keep the same order in the pair of values when entering the data. Here Jack was born in 1961 and Jill in 1964. Jack hopes to live to 89 years old, while Jill thinks she might reach age 92.

In [ ]:
plan = rp.Plan(YOB=[1961, 1964], expectancy=[89, 92])

## Specify account balances and spousal beneficiaries
For each spouse, savings accounts have three buckets comprising of the total value of:
- Individual taxable investment or savings accounts, including bank accounts, and CDs - do not include your safety net account which should typically be sufficient for sustaining 6 months of living expenses
- Tax-deferred savings accounts, including all IRAs, 401k, 403b, etc.
- Tax-free savings accounts, including Roth IRAs and Roth 401k

For married couples, each spouse will have values for each type of savings account, following the same order as before. For single individuals, only one value is needed between each paitr of brackets [ ].

Most investment accounts have named beneficiaries. The beneficiary values specify the fraction of total of assets left to the other spouse at death. For example, a spouse leaving 3/4 of her fortune to her three children and the other part to her partner would have a beneficiary value of 0.25.

Jack and Jill leave everything to each other as account beneficiaries as entered by [1, 1]. Jack has \\$90k in his taxable account, \\$500k in his 401k, and \\$80 in his Roth 401k and Roth IRAs. Jill has \\$40k in her savings bank account, \\$150k in a 403b, and \\$25k in Roth IRAs in which she contributed over the years.

In [ ]:
plan.setAssetBalances(taxable=[90000, 40000], 
                      taxDeferred=[500000, 150000],
                      taxFree=[80000, 25000],
                      beneficiary=[1., 1.])

## What are current and future asset allocations?
Each savings account can invest in 4 major classes of assets:
- Equity funds tracking the S&P500 index
- Bond assets tracking the Corporate bonds (Baa) index
- Fixed-income securities represented by the performance of 10-year Treasury bonds
- Inflation-indexed securities tracking urban Consumer Price Index (common assets).

The total of percentages in each class of assets for each savings account must add to 100%.

You are asked to provide an assets ratio distribution for today, and one for the end of your life.
Values in between will be interpolated using a linear operator (for now). This can be useful
if you want to shift assets allocation as you age.

In the example below, Jack's and Jill's allocations start in full stock equities in the tax-free account,
gradually transitioning to a traditional 60/40 portfolio towards the end of life. Their tax-deferred
account stays as a traditional 60/40 for the duration of their lives. Asset ratios are entered in percentages.

In [ ]:
plan.setInitialAR(taxableAR=[[0, 25, 50, 25], [0, 25, 50, 25]],
                  taxDeferredAR=[[60, 40, 0, 0], [60, 40, 0, 0]],
                  taxFreeAR=[[100, 0, 0, 0], [100, 0, 0, 0]])

plan.setFinalAR(taxableAR=[[0, 25, 50, 25], [0, 25, 50, 25]],
                taxDeferredAR=[[60, 40, 0, 0], [60, 40, 0, 0]],
                taxFreeAR=[[60, 40, 0, 0], [60, 40, 0, 0]])

plan.interpolateAR('linear')

## Specify rates of return and inflation rate
Rates of return can be specified for each class of assets, and inflation.
Valid entries are 'historical', 'stochastic', or a list of fixed average values.

For *historical* and *stochastic* data, data from 1928 to last year are available for experimenting.
Ranges chosen smaller that the life horizon will have rate values repeated in cycle. For example,
choosing historical data from 1994 to 1996 will repeat these three values over the time span of the realization.
This would be called as follows

    plan.setRates('hitorical', 1994, 1996)

If the last time bound is not provided as the third argument, then the latest data year (i.e., last year) will be assumed by default.
If one chooses a historical range starting from 1970, HARP will use the rates of 1970 for this year and 1971 for next, etc. This would be simulated as follows:

    plan.setRates('historical', 1970)
  
Due to its particular sequence of rates, the worst-case known historical scenario is a retirement starting in 1966. It can be simulated as follows:

    plan.setRates('historical', 1966)

In this case, the  current year will have the same rates as those that happened in 1966, and next year will have those from 1967, and so on. This is given for instructional purposes only. No one should plan using the worst-case scenario. But is can be informative to test your own case. In practice, a success rate larger then 90% over a reasonable set of market assumptions should be acceptable. But this is a personal choice.

On the other hand, stochastic data computes the multivariate distributions for the 4 rates in the selected year range and uses the computed distribution to generate random rate values. For example,

    plan.setRates('stochastic', 1945)
    
will analyze the data from 1945 to the last year and compute means and covariance to generate statistically similar results. The rates randomly generated for the timespan can be plotted and examined as we will see below. Similarly,

    plan.setRates('stochastic', 1940, 1970)

would generate random rates consistent to what was observed during the 1940 - 1970 time period.
    
Finally one can also use fixed annual rates by providing a list of 4 entries as follows:

    myrates = [9.6, 4.0, 3.0, 3.8]
    plan.setRates(myrates)
    
This example would use fixed average annual rates of 9.6%, 4% and 3% average annual returns on S&P500, corporate bonds, Treasury bonds, and common assets, respectively, with an average annual inflation rate of 3.8%. Note gain that common assets are investments tracking inflation only.

Also note that the S&P500 rates provided always include dividends, which are assumed to be reinvested.

Jill is interested to know if his situation would survive a retirement started in 1969.

In [ ]:
plan.setRates('historical', 1969)

## How much net income is desirable at retirement?
For determining this value, retirement planners will strongly suggest that you've must have done a cashflow analysis on your yearly spending. After this exercise, you should have a good idea of how much you'll need in retirement. Another approach is to experiment with multiple spending scenarios and see what your current and future savings can sustain under different market conditions.

The desired income defined here is the minimum net income (i.e., after paying taxes) that one would like to have starting at her/his "retirement age" (we will provide a loose definition the term *retirement age* below). This desired income must be adjusted for inflation and can follow an additional adjustment called a *smile* profile. A *smile* profile accounts for the fact that your spending capacity will modulate during retirement as you go from the so-called gogo years to the no-go years. A *flat* profile, on the other hand, will keep the same value, which will only be inflation-adjusted.

The target and actual net income values achieved by the realization can be plotted as will see below.

Jack and Jill believe that their spending profile will follow a *smile* curve rather than a *flat* curve. They also believe that they can be comfortable with \\$80k per year.   Can they afford it?

In [ ]:
plan.setDesiredIncome(80000, 'smile')

## What about anticipated fixed income?
Pension and social security are fixed income. Model here assumes that pension income is not inflation adjusted while social security benefits are (but HARP Labs can easily be modified to account for inflation-adjusted pensions). Entries are the predicted amount for each spouse and the age of the commencement of benefits.

If there are no pension benefits, just use zeros (0) as entries, as in

    plan.setPension([0, 0], [65, 65])
    
For social security, one must provide the predicted amount(s) and the starting age(s) at which benefits are received. There are plenty of social security benefit estimators on the web, including the info you can get directly from your own account at the Social Security Administration (ssa.gov).

Here, Jill has an unindexed pension of \\$10k per year. Both Jack and Jill believe they have good genes and decided to take their social security benefits at age 70. The amounts provided are estimation of the amounts they would receive at age 70. 

In [ ]:
plan.setPension([0, 10000], [65, 65])

plan.setSocialSecurity([30000, 28000], [70, 70])

## There must be a plan for savings
The most controllable part of retirement planning is the control one has over work income, contributions to savings accounts, Roth conversions, and other big spending items in the near- and mid-term future.
In order to execute a realization, one must provide an earning, saving, and Roth conversion plan. This is done through providing an Excel workbook with one spreadsheet (tab) per spouse with the following information:

|year|anticipated income|ctrb taxable | ctrb 401k | ctrb Roth 401k | ctrb IRA | ctrb Roth IRA | Roth X | big ticket items|
|--|--|--|--|--|--|--|--|--|
|2023 | | | | | | | | |
|2024 | | | | | | | | |
| ... | | | | | | | | |
|20XX | | | | | | | | |

Here, 20XX is the first year after both spouses have passed. For the columns, *anticipated income* is the annual amount (gross) that you anticipate to receive from employement or other sources (not including dividends from your taxable investment account). Note that column names are case sensitive and all of these entries must be in lower case. Best way to start is to use the template provided rightly named *template.xlsx*.

For the purpose of this exercise, there is no clear definition of retirement age. There will be a year, however, from which you will stop having anticipated income, or diminished income due to decreasing your work load. This transition can be gradual or sudden. Therefore there is no need to enter a retirement age for the sole purpose of quantifying your financial future.

Contributions to your savings accounts are marked as *ctrb*. Contributions to your 401k must also include your employer's contributions. As this file is in excel, one can use the calculator to enter a percentage of the anticipated income as this can sometimes be easier.

Roth conversion are specified in the column marked *Roth X*. Roth conversion are typically performed in the years when the income is lower (and therefore lower tax rates), typically in the bridge years between having a full-time regular salary and collecting social security. Finally, *big ticket items* are used for accounting for the sale or purchase of a house, or any other major expense or money that you would give or receive (e.g., inheritance, or large gifts to or from you). Therefore, the sign (+/-) of entries in this column is important. All other column entries should be positive.

The tab name for each spreadsheet represents the name of the spouse for reporting yearly transactions affecting your plan. So for Jack and Jill, this worksheet file needs two tabs, the first one named *Jack* and the second one named *Jill*, following the same order of data provided before. In fact, HARP Labs uses the names of these tabs to determine the individuals' names.

Note that the file format from the (free) LibreOffice software can also be read, so you do not need to have an Excel license.

Jack and Jill has provided their specific information in the file *jack+jill.xlsx*. Open this file in Excel and familiarize yourself with its contents.

In [ ]:
plan.readContributions('jack+jill.xlsx')

## How to withdraw from spousal accounts?
It can be desirable for couples to determine how to make distributions from each spousal account.
By specifying an *auto* spousal split, the withdrawals from the retirement savings accounts will be made proportial to their respective balances. It can also be specified as a fixed fractional value (e.g., 0.65) in which case 0.65 will be taken from the first account and the other 35% taken from the other (second entry) spouse.

All withdrawals use a smart banking approach which favors depleting taxable accounts before tax-deferred, before tax-free. With *auto*, additional checks and bounds are used to better coordinate the spousal accounts. Jack and Jill decide to rely o HARP to make the right decision on their behalf.

In [ ]:
plan.setSpousalSplit('auto')

## Run calculations
We're ready!

This final call runs all the required calculations for the time horizon set by the life expectancies of Jack and Jill.

In [ ]:
plan.run();

# Analysis

### Show net income compared to target income over the years
This graph shows how the actual net income generated by the plan realization matches the inflation-adjusted net income profile specified.

In [ ]:
plan.plotNetIncome();

### Show sources of income over the years
Income will be typically coming from mutiple sources, and it can be quite complex. This graphs shows the breakdown of Jack's and Jill's sources of income by spouse and by origin. Note that distributions from tax-deferred accounts (*dist*) are distinguished from required minimum distributions (*rmd*) as they serve different purposes. Other labels should be self-expanatory.

In [ ]:
plan.plotSources();

### Show savings accounts balances at the beginning of each year
The balance for each savings account for each spouse is calculated at the beginning of each year. Another important aspect of this graph is how much is left at the end of the realization. This will be addressed in the next cell.

In [ ]:
plan.plotAccounts();

To get more information about the last year of the realization, one can use

    plan.estate(30)
    
which returns the value of the estate, assuming (in this case) a 30\% tax burden on the taxable portion of the estate (read tax-deferred savings accounts). The `estate()` function returns two values: the total post-tax value of all savings account in today's dollars and the cumulative inflation rate between today and the last day of the realization. Here we also use utility functions to properly display dollar (`rp.d()`) and percentage (`rp.pc()`) values provided by the HARP module.

In [ ]:
total, percent = plan.estate(30)
print(rp.d(total), rp.pc(percent))

### Show annual taxes paid over the years of the realization
This graph shows how much Jack and Jill paid in federal taxes and IRMAA income-related Medicare insurance monthly adjustments over the years of this realization.

In [ ]:
plan.plotTaxes();

### Show taxable annual income and anticipated tax brackets
Gross income also includes Roth conversions and big-ticket items, both of which are not contributing to your net income. This graph shows Jack and Jill's gross taxable income and how it compare with some anticipated Federal tax marginal brackets. This visualization is very convenient when one wants to perform Roth conversions and remain below a certain tax bracket.

Note the shift in tax brackets taking place as the Tax Cut and Job Act expires after 2025.

In [ ]:
plan.plotTaxableIncome();

### Show annual rates used for calculations
As described above, there are many choices for selecting rates. This graph will display the annual rates used during the time span of this realization.

In [ ]:
plan.plotRates();

## Do we save this realization?
This realization contains information on the distribution amounts, including the required minimum distribution that had to be performed under the given assuptions. This info can be saved in an excel workbook with one spreadsheet (tab) for Jack and one for Jill.

In [ ]:
# plan.saveRealizationXL('Jack and Jill')

## Next level
The flexibility of HARP Labs allows for scripting Monte-Carlo simulations by embedding the calls described above in an iterative script. In those simulations, multiple instances are executed to further explore the robustness of one's decisions under different market realizations.

The script *jack+jill.py* is intended for that purpose and is provided as an example. It showcases two possibilities. In the first one, historical data is used to explore how the plan survives the test of historical rates from 1928 to the most recent past. The other capability shows a Monte-Carlo approach to generate ficticious rates derived from the statistical distribution of those observed in a selected time period.

This script is a template for designing your own. It can be run through the command line, but they can also be run from jupyter using the %run function. Let' delve into these capabilities.


### Historical comparisons
When run with the *historical argument*, the script runs Jack's and Jill's scenario through historical data starting from 1930 to 1986. Note how their planned scenario is successful in all years except 1966 and 1969 if they are willing to spend \\$79,500 as inflation-adjusted net income. 

In [ ]:
%run jack+jill.py historical

### Monte-Carlo simulations
In a Monte-Carlo simulation, a random number generator is used to create values that meet the statistical criterion of a known distribution. In HARP Labs, one specifies the range of years of historical data that will be used to characterize the probability distribution of the normal distribution. Once selected, one can run as many realization as desired. The more is generally the better, as it will improve the mean while increasing the possibility of observing rare events. A value of 1,000 is generally sufficient. Open the script to see what range of years were used to derive the statistical distribution.

In [ ]:
%run jack+jill.py Monte-Carlo 1000